In [1]:
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [64]:
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
                                                    train_size=0.75, test_size=0.25)

tpot = TPOTClassifier(generations=10, population_size=50, verbosity=2, n_jobs=4)

In [65]:
%%time

tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_mnist_pipeline.py')

Generation 1 - Current best internal CV score: 0.9828562821946406


Generation 2 - Current best internal CV score: 0.9828562821946406


Generation 3 - Current best internal CV score: 0.9828562821946406


Generation 4 - Current best internal CV score: 0.9836771016767235


Generation 5 - Current best internal CV score: 0.9843930933158891


...

Generation 7 - Current best internal CV score: 0.9881275235058107


Generation 8 - Current best internal CV score: 0.9881275235058107


Generation 9 - Current best internal CV score: 0.9881275235058107


Generation 10 - Current best internal CV score: 0.9903303230886917



Best pipeline: KNeighborsClassifier(Normalizer(input_matrix, Normalizer__norm=l2), KNeighborsClassifier__n_neighbors=2, KNeighborsClassifier__p=DEFAULT, KNeighborsClassifier__weights=distance)
0.975555555556
CPU times: user 45.5 s, sys: 1.07 s, total: 46.6 s
Wall time: 22min 24s


In [66]:
! cat tpot_mnist_pipeline.py

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

# NOTE: Make sure that the class is labeled 'class' in the data file
tpot_data = np.recfromcsv('PATH/TO/DATA/FILE', delimiter='COLUMN_SEPARATOR', dtype=np.float64)
features = np.delete(tpot_data.view(np.float64).reshape(tpot_data.size, -1), tpot_data.dtype.names.index('class'), axis=1)
training_features, testing_features, training_target, testing_target = \
    train_test_split(features, tpot_data['class'], random_state=42)

exported_pipeline = make_pipeline(
    Normalizer(norm="l2"),
    KNeighborsClassifier(n_neighbors=2, weights="distance")
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


In [59]:
import h2o

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_144"; Java(TM) SE Runtime Environment (build 1.8.0_144-b01); Java HotSpot(TM) 64-Bit Server VM (build 25.144-b01, mixed mode)
  Starting server from /home/yannick/bin/anaconda3/envs/py36/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp2mguajl3
  JVM stdout: /tmp/tmp2mguajl3/h2o_yannick_started_from_python.out
  JVM stderr: /tmp/tmp2mguajl3/h2o_yannick_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster version:,3.14.0.2
H2O cluster version age:,25 days
H2O cluster name:,H2O_from_python_yannick_qzlkic
H2O cluster total nodes:,1
H2O cluster free memory:,3.476 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [61]:
target = "C1"

import numpy as np

def to_h2o(X, y):
    X_h2o = h2o.H2OFrame(np.concatenate((y[:,None], X), axis = 1))
    X_h2o[target] = X_h2o[target].asfactor()
    return X_h2o

X_train_h2o = to_h2o(X_train, y_train)
X_test_h2o = to_h2o(X_test, y_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [68]:
%%time 

from h2o.automl import H2OAutoML

aml = H2OAutoML(max_runtime_secs=1200)

aml.train(x=X_train_h2o.columns.remove(target),
          y=target,
          training_frame=X_train_h2o,
          leaderboard_frame=X_test_h2o)

aml.leaderboard

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
CPU times: user 5.13 s, sys: 264 ms, total: 5.4 s
Wall time: 20min 5s


In [69]:
aml.leaderboard

model_id,mean_per_class_error
GBM_grid_1_AutoML_20170916_154811_model_7,"0,014110"
GBM_grid_1_AutoML_20170916_163753_model_1,"0,016491"
GBM_grid_1_AutoML_20170916_163753_model_9,"0,018881"
GBM_grid_1_AutoML_20170916_163753_model_11,"0,019655"
GBM_grid_1_AutoML_20170916_163753_model_22,"0,020734"
GBM_grid_0_AutoML_20170916_163753_model_2,"0,020790"
GBM_grid_1_AutoML_20170916_163753_model_5,"0,021091"
GBM_grid_1_AutoML_20170916_163753_model_27,"0,022909"
GBM_grid_0_AutoML_20170916_163753_model_1,"0,023124"
GBM_grid_1_AutoML_20170916_163753_model_28,"0,024474"


In [67]:
1-0.0141

0.9859